# Genomic Data Processing and Creation of Polygenic Risk Score

### Sources:

> DNANexus Tutorial: https://www.youtube.com/watch?v=762PVlyZJ-U&t=2027s <br />
>
> PGS Catalog Scoring File Page: https://www.pgscatalog.org/score/PGS001780/ <br />
>
> https://github.com/ninamars/INTERVENE-PRS-transferability/blob/main/prs_calculation.sh <br />
>
> Tamlander M, Mars N, Pirinen M, Widén E, Ripatti S. Integration of questionnaire-based risk factors improves polygenic risk scores for human coronary heart disease and type 2 diabetes. Commun Biol 2022 51. 2022;5(1):1-13. doi:10.1038/s42003-021-02996-0
>
> Birling M-C, Yoshiki A, Adams DJ, et al. The Polygenic Score Catalog as an open database for reproducibility and systematic evaluation. Nat Genet 2021 534. 2021;53(4):420-425. doi:10.1038/s41588-021-00783-5
>
> Mitchell R, Hemani G, Dudding T, Corbin L, Harrison S, Paternoster L. UK Biobank Genetic Data: MRC-IEU Quality Control, version 2. https://doi.org/10.5523/bris.1ovaau5sxunp2cv8rcy88688v. Published 2019.


## Selecting Variants for PRS

This section involved the creation of a polygenic risk score to match the one used in Tamlander *et al.*, 2022 that was the most recent and powerful to date, as well as tested but not trained on UKB data, which makes it an ideal choice to avoid winner’s curse bias. I first downloaded the 22 ukb_mfi_chrX_v3.txt files from UK Biobank data showcase. These files contain minor allele frequencies and INFO (imputation quality) scores but for our current purposes the most relevant aspect of the files is that they contain all variants either assayed or imputed in each chromosome. I also downloaded the Scoring File from the PGS Catalog (ID: 001780), which contains all of the variants and the weights required to implement the polygenic score (and applied to the correct genomic build of GRCH37 - same as UKB). 


## Matching Variants in UKB and PRS

I next compare the ukb_mfi_chrX_v3.txt files with the variants listed in the Scoring File on PGS Catalog with the below R code by matching rsIDs. There were 1,089,716 matched variants (compared to 1,090,048 overall and 1,087,714 in the UKB after screening in Tamlander). 

NOTE: This process was conducted ENTIRELY offline in R because it relied entirely on publicly available, downloadable information. We show the code for only chromosome 1 for the sake of brevity but the process is the same for all 22 autosomes.

In [ ]:

# Reading in mfi file for chrom 1
CHR1UKBVARS <- read.table("ukb_mfi_chr1_v3.txt", colClasses = c("character", "character", "character", "character", "character", "character", "character", "character"))

# NOW putting UKB variants into the same format as in Scoring File (rsIDs)
CHR1UKBVARSRSIDS <- CHR1UKBVARS$V2

# Keeping Chr:Pos_Alleles and POS variable
CHR1UKBVARSPOS <- CHR1UKBVARS[ , c(1,3)]


# Removing everything after underscore for Chr:Pos_ variable - aligning with Scoring File
# This produces Chr:Pos variable
CHR1UKBVARSPOS$V1 <- gsub(pattern = "_.*", replacement = "", x = CHR1UKBVARSPOS$V1)

# NOW adding -POS again for Chr:Pos using POS variable
CHR1UKBVARSPOS$V1 <- paste(as.character(CHR1UKBVARSPOS$V1), as.character(CHR1UKBVARSPOS$V3), sep = "-")

# NOW keeping ONLY Chr:Pos-Pos variable
CHR1UKBVARSPOS <- CHR1UKBVARSPOS[ , 1]


In [ ]:
# --------
# NOW loading in Scoring File to compare
# --------

ScoringFile <- read.table("PGS001780GRCH37.txt", fill = TRUE, header = TRUE, colClasses = c("character", "character", "character", "character", "character", "character"))


# Some variants had missing rsIDs
# So first get subset w/ ONLY those w/o RSID and fill in missing RSIDs manually
ScoringFileNORSID <- ScoringFile[grepl("^[rs]", ScoringFile$rsID) == FALSE, ]
# 22 variables to manually insert rsid


# Missing rsIDs to insert
RSIDs <- c("rs6661843", "rs11247652", "rs3934842", "rs1511825", "rs4262688", "rs17109000", "rs4410035",
           "rs34233747", "rs9973046", "rs254279", "rs13430871", "rs11893266", "rs11694726", "rs13397261",
           "rs4513433", "rs17089780", "rs6844962", "rs2171193", "rs1678262", "rs219947", "rs3988302", "rs4571835")


# Getting column names correct - to merge with overall Scoring File
colnames(ScoringFileNORSID) <- c("chr_name", "chr_position", "effect_allele", "other_allele", "effect_weight", "variant_description", "blank")

# Creating rsID column
ScoringFileNORSID$rsID <- RSIDs

# Now drop 'blank' column and merge w/ the full Scoring File
ScoringFileNORSID <- ScoringFileNORSID[-7]

# EXCLUDING these NORSID variants BEFORE merge
ScoringFileRSID <- ScoringFile[grepl("^[rs]", ScoringFile$rsID) == TRUE, ]

# Merging RSID and NORSID datasets
ScoringFile <- rbind(ScoringFileNORSID, ScoringFileRSID)

# Confirming this merged as it should and full Scoring File now has rsIDs
summary(as.factor(ScoringFile$chr_name))
summary(as.factor(ScoringFile$rsID))


# ---
# Creating a Chr:Pos-Pos variable to merge with mfi file
# ----
# Creating Chr:Pos
ScoringFile$POS <- paste(as.character(ScoringFile$chr_name), as.character(ScoringFile$chr_position), sep = ":")

# NOW appending additional -Pos
ScoringFile$POS <- paste(as.character(ScoringFile$POS), as.character(ScoringFile$chr_position), sep = "-")


# Subset scoring file to only relevant chromosome
ScoringFileCHROM1 <- subset(ScoringFile, chr_name == 1)


# NOW keeping ONLY RSIDs for Scoring File to compare
ScoringFileCHROM1RSONLY <- ScoringFileCHROM1[c(7)]



In [ ]:

# --------
# Officially checking where Scoring File and mfi file match and do not match
# --------

dim(ScoringFileCHROM1RSONLY)
dim(as.data.frame(CHR1UKBVARSRSIDS))
# 90,435 variants in PRS
# 7,402,791 variants in UKB


summary(match(ScoringFileCHROM1RSONLY$rsID, CHR1UKBVARS$V2))
summary(match(CHR1UKBVARS$V2, ScoringFileCHROM1RSONLY$rsID))
# 1412 NAs 

# Keeping only rows where Scoring File and mfi from UKB match
ROWSKEEP <- na.omit(match(ScoringFileCHROM1RSONLY$rsID, CHR1UKBVARS$V2))


# NOW have an RSIDS vector that ONLY includes RSIDs in BOTH UKB and PRS
UPDATEDRSIDS <- CHR1UKBVARS[ROWSKEEP, ]
# 90,435 variants in chromosome 1 are in both Scoring File and UK Biobank


UPDATEDRSIDS <- as.data.frame(UPDATEDRSIDS)

# Save as txt file
write.table(UPDATEDRSIDS, "UPDATEDRSIDS1NEWSCORE.txt")


# Repeated this process for all 22 autosomes to yield 1,090,048 variants

## Quality Control Restricting of Matched Variants

After matching variants between the UK Biobank and Scoring File for the Tamlander *et al.* PRS, we next dropped variants that were either multi-allelic or had an INFO score of less than or equal to 0.6 (denoting relatively low imputation quality). We got the INFO score and information on multi-allelic SNPs from the publicly available files in the UK Biobank Showcase. This process reduces the number of variants only marginally to 1,089,653.

In [ ]:

# -----
# Removing low IFNO score variants
# -----

# Loading in mfi file for INFO score
CHR1UKBVARS <- read.table("ukb_mfi_chr1_v3.txt", colClasses = c("character", "character", "character", "character", "character", "character", "character", "character"))


# Loading in variants selected earlier
CHROM1RSID <- read.table("UPDATEDRSIDS1NEWSCORE.txt")


# Final column of mfi file is INFO score, so restrict to over 0.6
CHR1UKBVARSINFO <- subset(CHR1UKBVARS, CHR1UKBVARS$V8 > 0.6)

# Only keep variants from matched file earlier that meet INFO score criteria
ROWSKEEPPOS1 <- na.omit(match(CHROM1RSID$V2, CHR1UKBVARSINFO$V2))

# NOW have an RSIDS vector that ONLY includes RSIDs in BOTH UKB and PRS
# For chromosome 1 this removes no variants
UPDATEDRSID1 <- CHR1UKBVARSINFO[ROWSKEEPPOS1, ]
# 90,435 variants


# -----
# Removing multi-allelic variants
# -----

# Loading in csv with multi-allelic SNPs listed
multi <- read.csv("multiallelicsnps.csv", colClasses = c("character", "character", "character", "character", "character"))


# Adding rs to match w/ RSID of matched variant file
multi$RSID <- paste("rs", as.character(multi$X109200108), sep = "")

# Remove missing rows
multi <- na.omit(multi)

# Compare mulit-allelic SNPs and matched variant file
summary(match(multi$RSID, UPDATEDRSID1$V2))
# 7093 NAs - makes sense (means there are 10 multi-allelic SNPs)

# Dropping variants identified as multi-allelic
DROPMATCH1 <- as.numeric(na.omit(match(multi$RSID, UPDATEDRSID1$V2)))

UPDATEDRSID1 <- UPDATEDRSID1[-DROPMATCH1, ]
# NOW 90,432 variants


# Some of V1 is mislabeled using RSID instead of Chr:Pos
summary(grepl("^[rs]", UPDATEDRSID1$V1, ignore.case=TRUE))
# Getting feel for scope of the issue - 42,376 instances of this out of all variants
# Solution is to recreate POS variable


# NOW keeping only rsIDs and removing column name
UPDATEDRSID1RSIDs <- UPDATEDRSID1$V2

# Creating and saving POS-POS for plink
UPDATEDRSID1$POS <-  paste("1", as.character(UPDATEDRSID1$V3), sep = ":")
UPDATEDRSID1$POS <-  paste(as.character(UPDATEDRSID1$POS), as.character(UPDATEDRSID1$V3), sep = "-")

UPDATEDRSID1POS <- UPDATEDRSID1$POS


colnames(UPDATEDRSID1RSIDs) <- NULL
rownames(UPDATEDRSID1RSIDs) <- NULL

colnames(UPDATEDRSID1POS) <- NULL
rownames(UPDATEDRSID1POS) <- NULL

# Saving rsID and Chr:Pos-Pos versions of chromosome 1
write.table(UPDATEDRSID1RSIDs, "FINALRSIDsChrom1NEWSCORERSIDS.txt", col.names = F, row.names = F, quote = FALSE)
write.table(UPDATEDRSID1POS, "FINALRSIDsChrom1NEWSCOREPOS.txt", col.names = F, row.names = F, quote = FALSE)


# Clear workspace - high memory usage, so clear each time
rm(list = ls())

# Repeating this process for all 22 autosomes yields 1,089,653 variants

## Converting bgen files to bed format (bed/bim/fam)

Imputed genetic variant files are in bgen format in the UK Biobank, which is difficult to use with most of the standard genomic analysis software. Thus, I next converted the chromosome files from bgen to bed format (a standard plink binary file format that produces 3 files - bed/bim/fam). The unfortunate part about bed files that is not true of pgen files (the newer binary file used in plink) is that it does not keep track of dosage information and instead hard codes. That is, instead of storing probabilities it makes these imputations look certain with some threshold chosen. This ultimately does not appear to substantially affect PRS performance. This code produces bed files NEWCHRXXBED.bed (from 1 to 22).

NOTE: Ran these commands in Swiss Army Knife.

In [ ]:
# Created command to be used by QC Tools to generate bed files from bgen files
QCTRY="qctool -g /mnt/project/Bulk/Imputation/UKB\ imputation\ from\ genotype/ukbXX_c22_b0_v3.bgen -s /mnt/project/Bulk/Imputation/UKB\ imputation\ from\ genotype/ukbXX_cXX_b0_v3.sample -incl-rsids /mnt/project/FINALRSIDsChromXXNEWSCORERSIDS.txt -ofiletype binary_ped -og NEWCHRXXBED"

dx run swiss-army-knife \
-icmd="${QCTRY}" --brief --yes
# Ran separately for all 22 autosomal chromosome pairs
# Produces bed/bim/fam files titled NEWCHRXXBED.bed from 1 to 22

## Restricting to minor allele frequency > 0.01

We next restricted to variants that had a minor allele frequency > 0.01, which is a fairly typical standard and screens out any variants with extremely low levels of variation. This is a higher threshold than in Tamlander *et al.*, which only removed variants with minor allele counts < 3.

NOTE: Plink2 was run in Swiss Army Knife

In [ ]:
# Restricts to maf > 0.01
MAFTEST="plink2 --bfile /mnt/project/NEWCHRXXBED --maf --make-bed --out BED1MAFREST"


dx run swiss-army-knife \
-icmd="${MAFTEST}" --brief --yes


##  Comparing Alleles to Matched Scoring File

We next ensured that variant alleles matched those in the Scoring File. We did this by reading the .bim file of the MAF restricted bed file into R and looking for mismatches with the Scoring File created earlier. We then flipped variants that did not match the alleles in the Scoring File. If these variants still did not match, they were removed from the score (only a few dozen such variants in the entire sample). There are now 1,087,647 variants (2,069 variants fewer than in last checkpoint  owing almost entirely to MAF screening). Our more stringent inclusion criteria yielded 67 fewer variants than in the Tamlander *et al.* score for the UK Biobank, which does not appear to significantly impact performance.

In [ ]:
# In R

# Reading in bim file and checking whether alleles are aligned
MAFCHECK <- read.table("/BED1MAFREST.txt")

# Determining whether alleles are aligned between Scoring File and UKB
MAFCHECK$POS1 <- paste(as.character(MAFCHECK$V2), as.character(MAFCHECK$V6), as.character(MAFCHECK$V5), sep = "_")
MAFCHECK$POS2 <- paste(as.character(MAFCHECK$V2), as.character(MAFCHECK$V5), as.character(MAFCHECK$V6), sep = "_")

# Remove if neither POS matches Scoring File
MATCH1 <- as.numeric(na.omit(match(ScoringFileChrom1$POS, MAFCHECK$POS1)))
MATCH2 <- as.numeric(na.omit(match(ScoringFileChrom1$POS, MAFCHECK$POS2)))

MATCHALL <- c(MATCH1, MATCH2)

# Isolating these missing values - ONLY those that do not match in EITHER
MAFCHECKALIGNED <- MAFCHECK[MATCHALL, ]

# Creating list of IDs w/ apparent strand misalignment
MAFCHECKMISMATCH <- MAFCHECK[-MATCHALL, ]
# 9 total that fit this criteria

# Isolate RSIDs and add to next plink command flipping alleles
write.table(MAFCHECKMISMATCH$V2, "FlippedSNPsChr1.txt", col.names = F, row.names = F, quote = FALSE)


In [ ]:
# In Swiss Army Knife
FLIP="plink --bfile /mnt/project/BED1MAFREST --flip /mnt/project/FlippedSNPsChr1.txt --make-bed --out BED1MAFRESTFlip"


dx run swiss-army-knife \
-icmd="${FLIP}" --brief --yes


In [ ]:
# In R

# ----------
# Checking if this all worked to align scoring file and alleles in chromosome 1
# ----------

CHROM1 <- read.table("/BED1MAFRESTFlip.txt")


# Have to make both since could differ w/ scoring file in either direction
CHROM1$POS1 <- paste(as.character(CHROM1$V2), as.character(CHROM1$V6), as.character(CHROM1$V5), sep = "_")
CHROM1$POS2 <- paste(as.character(CHROM1$V2), as.character(CHROM1$V5), as.character(CHROM1$V6), sep = "_")


summary(match(ScoringFileChrom1$POS, CHROM1$POS1))
summary(match(ScoringFileChrom1$POS, CHROM1$POS2))


MATCH1 <- as.numeric(na.omit(match(ScoringFileChrom1$POS, CHROM1$POS1)))
MATCH2 <- as.numeric(na.omit(match(ScoringFileChrom1$POS, CHROM1$POS2)))

MATCHALL <- c(MATCH1, MATCH2)

# Isolating these missing values - ONLY those that do not match in EITHER
CHROM1ALIGNED <- CHROM1[MATCHALL, ]
# 68093

# Creating list of IDs w/ strand flipping
CHROM1MISMATCH <- CHROM1[-MATCHALL, ]


# Isolate RSIDs and add to next plink command if still no match with Scoring File
write.table(CHROM1MISMATCH$V2, "CHROM1MISMATCH.txt", col.names = F, row.names = F, quote = FALSE)


In [ ]:
FINAL="plink --bfile /mnt/project/BED1MAFRESTFlip --exclude /mnt/project/CHROM1MISMATCH.txt --make-bed --out BED1FINAL"

dx run swiss-army-knife \
-icmd="${FINAL}" --brief --yes

## Merging bed files into single file

We merged all of the bed files into a single binary file (bed/bid/fam) using plink.

In [ ]:

run_merge="cp /mnt/project/BEDFINAL[1-9]* . ;\
        ls *.bed | sed -e 's/.bed//g'> files_to_merge.txt; \
        plink --merge-list files_to_merge.txt --make-bed\
        --autosome-xy --out BEDFINAL_1_22_merged;\
        rm files_to_merge.txt;"


dx run swiss-army-knife \
   -icmd="${run_merge}" --tag="Step1" --instance-type "mem1_ssd1_v2_x36"\
   --brief --yes


## Fitting the Polygenic Risk Score

I finally fit the polygenic score using plink, which is compared to Tamlander, *et al.* later.

In [ ]:
# In Swiss Army Knife
plinkpgs="plink2 --bfile /mnt/project/BEDFINAL_1_22_merged \
--score /mnt/project/PGS001780GRCH37.txt 1 4 6 header list-variants cols=scoresums \
--out FINALPGS"

dx run swiss-army-knife \
-icmd="${plinkpgs}" --brief --yes


The above process yielded the polygenic risk score for all individuals with genomic data based on the weights from the Scoring File.